In [22]:
%%writefile employee.py

class Employee:

  def __init__(self, first, last, pay, ident):
    self.first = first
    self.last = last
    self.pay = pay
    self.ident = ident

  @property
  def fullname(self):
    return f'{self.first} {self.last}'

  @property
  def email(self):
    return f'{self.first.lower()}.{self.last.lower()}@company.com'

  def __repr__(self):
    return f'Employee({self.first}, {self.last}, {self.pay}, {self.ident})'

Overwriting employee.py


In [23]:
import sqlite3
from employee import Employee

In [24]:
# Create connecction (even it exists)
# conn = sqlite3.connect('employee.db')

# Or in memory (good for testing)
conn = sqlite3.connect(':memory:')

In [25]:
# Create cursor, so we can run sql command
c = conn.cursor()

In [26]:
# Create employee table (throw error if exists)
# https://www.sqlite.org/datatype3.html
c.execute("""CREATE TABLE employees (
  first TEXT,
  last TEXT,
  pay INTEGER,
  ident TEXT
)""")

# Commit changes
conn.commit()

In [27]:
def insert_emp(emp):
  with conn:
    c.execute("INSERT INTO employees VALUES (:first, :last, :pay, :ident)",
      {'first': emp.first, 'last': emp.last, 'pay': emp.pay, 'ident': emp.ident})

def get_emps_by_name(lastname):
  if lastname == None:
    c.execute("SELECT * FROM employees")
  else:
    c.execute("SELECT * FROM employees WHERE last=:last", {'last': lastname})
  return c.fetchall()

def update_pay(emp, pay):
  with conn:
    c.execute("""UPDATE employees SET pay = :pay
      WHERE first = :first AND last = :last""",
      {'first': emp.first, 'last': emp.last, 'pay': pay})

def remove_emp(emp):
  with conn:
    c.execute("DELETE from employees WHERE first = :first AND last = :last",
      {'first': emp.first, 'last': emp.last})

In [28]:
emp_1 = Employee('John', 'Doe', 80000, '001')
emp_2 = Employee('Jane', 'Doe', 90000, '002')
emp_3 = Employee('Corey', 'Schafer', 100000, '003')
emp_4 = Employee('Tom', 'Smith', 110000, '004')

In [29]:
insert_emp(emp_1)
insert_emp(emp_2)
insert_emp(emp_3)
insert_emp(emp_4)

In [30]:
emps = get_emps_by_name('Doe')
print(emps)

[('John', 'Doe', 80000, '001'), ('Jane', 'Doe', 90000, '002')]


In [31]:
update_pay(emp_2, 95000)

In [32]:
remove_emp(emp_1)

In [33]:
emps = get_emps_by_name(None)
print(emps)

[('Jane', 'Doe', 95000, '002'), ('Corey', 'Schafer', 100000, '003'), ('Tom', 'Smith', 110000, '004')]


In [34]:
conn.close()